In [30]:
import tensorflow as tf
import numpy as np
import os


In [31]:
Null_TAG = 'None'
P_TAG_b = 'Pb'  # beginning of participant phrase
P_TAG_m = 'Pm'  # middle/end of participant phrase

In [86]:
def annotate_abstract(abstract_path, gold_annotation_path):
    abs_file = open(abstract_path, 'r');
    file_text = abs_file.read();

    ann_file = open(gold_annotation_path, 'r');
    ann_file = ann_file.read();

    ann_list = ann_file.split();
    part_list = [];
    for i in range(1, len(ann_list), 2):
        part_list.append((int(ann_list[i]), int(ann_list[i+1])))


    word_list = file_text.split(); # [word1, word2, word] no spaces
    tag_list = []
    index = 0; # changed from -1
    ann_index = 0
    if (len(part_list) == 0):
        ann_start = np.inf
        ann_end = np.inf
    else:
        ann_start = part_list[ann_index][0]
        ann_end = part_list[ann_index][1]
    in_phrase = False
    sentence_start_ind = [0]

    for word_ind in range(len(word_list)):
        word = word_list[word_ind]
        index += len(word) + 1;
        if not in_phrase:
            # looking for start of participant phrase
            if (ann_start < index):
                # we found first word in this participant segment
                tag_list.append(P_TAG_b)
                in_phrase = True
            else:
                tag_list.append(Null_TAG)          
        else:
            tag_list.append(P_TAG_m)
            # in the participant phrase, looking for its end
            if (ann_end <= index):
                # we found the last word in the participant segment
                ann_index += 1
                if (ann_index == len(part_list)):
                    ann_start = np.inf
                    ann_end = np.inf
                else:
                    ann_start = part_list[ann_index][0]
                    ann_end = part_list[ann_index][1]
                in_phrase = False
        if word[-1] == ".":
            sentence_start_ind.append(word_ind+1)
    
    if not (len(word_list) == len(tag_list)):
        print "FUCK"
        raise ValueError('For this file, len of abstract words and tags did not match.', abstract, len_abs, len_tags)

    out_abs_path = abstract_path[0:-4] + '_input.txt'
    out_ann_path = abstract_path[0:-4] + '_input_tags.ann'
    
    abs_out_str = ''
    ann_out_str = ''
    
    for i in range(0, len(sentence_start_ind)-1):        
        sentence = ' '.join(word_list[sentence_start_ind[i]:sentence_start_ind[i+1]])
        tag_sentence = ' '.join(tag_list[sentence_start_ind[i]:sentence_start_ind[i+1]])
        
        if not (len(sentence) == len(tag_sentence)):
            print "FUCK"
            raise ValueError('For this file, len of abstract words and tags did not match.', abstract_path)

        
        abs_out_str = abs_out_str + sentence + '\n'
        ann_out_str = ann_out_str + tag_sentence + '\n'
    
    abs_f = open(out_abs_path, 'w')
    ann_f = open(out_ann_path, 'w')
    
    abs_f.write(abs_out_str);
    ann_f.write(ann_out_str);
    
    abs_f.close();
    ann_f.close();
    

In [87]:
abs_path = 'PICO-annotations/batch5k/0074f5e102cf4409ac07f6209dd30144/20957980.txt'
ann_path = 'PICO-annotations/batch5k/0074f5e102cf4409ac07f6209dd30144/20957980_gold.ann'
annotate_abstract(abs_path, ann_path)

FUCK


ValueError: ('For this file, len of abstract words and tags did not match.', 'PICO-annotations/batch5k/0074f5e102cf4409ac07f6209dd30144/20957980.txt')

In [37]:
directory = 'PICO-annotations/batch5k'

# For each subdirectory
for subdir in os.listdir(directory):
    subdir_path = directory + '/' + subdir
    # print subdir_path
    
    # Not a directory
    if not os.path.isdir(subdir_path):
        continue
    
    # For each abstract in subdirectory
    for abstract in os.listdir(subdir_path):
        if (abstract.endswith('.txt')) and not (abstract.endswith('input.txt')):
            abstract_path = subdir_path + '/' + abstract; 
            # print abstract_path
            ann_path = abstract_path[0:-4] + '_gold.ann'
            annotate_abstract(abstract_path, ann_path)

In [58]:

def read_file(abstract_path, gold_annotation_path=None, seperate_sentances=False):    
    abstract_file = open(abstract_path, 'r');
    file_text = abstract_file.readlines();
    file_text = np.array([line.strip() for line in file_text])
    text_words_separated = np.array([line.split() for line in file_text])
    if not seperate_sentances:
        text_array = []
        [text_array.extend(x) for x in text_words_separated]
    else:
        text_array = text_words_separated
    abstract_file.close()

    if not(gold_annotation_path is None):
        tag_file = open(gold_annotation_path);
        tags = tag_file.readlines();
        tags = np.array([line.strip() for line in tags])
        tags_separated = np.array([line.split() for line in tags])
        if not seperate_sentances:
            tag_array = []
            [tag_array.extend(x) for x in tags_separated]
        else:
            tag_array = tags_separated
        tag_file.close()
    return [text_array, tag_array]
    

In [59]:
abs_path = 'PICO-annotations/batch5k/0074f5e102cf4409ac07f6209dd30144/20957980_input.txt'
ann_path = 'PICO-annotations/batch5k/0074f5e102cf4409ac07f6209dd30144/20957980_input_tags.ann'
[abs_array, tag_array] = read_file(abs_path, ann_path)


In [76]:
def get_all_data(data_directory='PICO-annotations/batch5k'):
    abstract_array = []
    tag_array = []
    num_files = 0; 
    num_bad = 0;
    
    # For each subdirectory
    for subdir in os.listdir(data_directory):
        subdir_path = directory + '/' + subdir
        # print subdir_path

        # Not a dir`ectory
        if not os.path.isdir(subdir_path):
            continue

        # For each abstract in subdirectory
        for abstract in os.listdir(subdir_path):
            if (abstract.endswith('input.txt')):
                num_files += 1;
                abstract_path = subdir_path + '/' + abstract; 
                annotation_path = abstract_path[0:-4] + '_tags.ann'
                # print abstract_path
                [curr_abs_array, curr_tag_array] = read_file(abstract_path, ann_path)
                len_abs = len(curr_abs_array)
                len_tags =  len(curr_tag_array)
                if not (len_abs == len_tags):
                    num_bad += 1
                    print 'Bad file: ', abstract_path
                    # raise ValueError('For this file, len of abstract words and tags did not match.', abstract, len_abs, len_tags) 
                abstract_array.extend(curr_abs_array)
                tag_array.extend(curr_tag_array)
    
    abstract_array = np.array(abstract_array)
    tag_array = np.array(tag_array)
    print 'num files: ', num_files
    print 'bad files: ', num_bad
    return [abstract_array, tag_array]

In [77]:
[word_array, tag_array] = get_all_data();
print word_array.shape
print tag_array.shape

Bad file:  PICO-annotations/batch5k/0074f5e102cf4409ac07f6209dd30144/23127528_input.txt
Bad file:  PICO-annotations/batch5k/0074f5e102cf4409ac07f6209dd30144/9665186_input.txt
Bad file:  PICO-annotations/batch5k/00a340c811a841f3bbf15f4828fd43dd/2182235_input.txt
Bad file:  PICO-annotations/batch5k/00a340c811a841f3bbf15f4828fd43dd/23773944_input.txt
Bad file:  PICO-annotations/batch5k/00a340c811a841f3bbf15f4828fd43dd/9732932_input.txt
Bad file:  PICO-annotations/batch5k/00f9eddfa8674d2e86b2f5c065685de0/11004058_input.txt
Bad file:  PICO-annotations/batch5k/00f9eddfa8674d2e86b2f5c065685de0/11246578_input.txt
Bad file:  PICO-annotations/batch5k/00f9eddfa8674d2e86b2f5c065685de0/21295919_input.txt
Bad file:  PICO-annotations/batch5k/0153faa6ed8f438ca815dd9827262a65/1715416_input.txt
Bad file:  PICO-annotations/batch5k/0153faa6ed8f438ca815dd9827262a65/17909199_input.txt
Bad file:  PICO-annotations/batch5k/0153faa6ed8f438ca815dd9827262a65/2015394_input.txt
Bad file:  PICO-annotations/batch5k/0

In [ ]:
print num